In [2]:
import json
with open("data/train.json", 'r', encoding='utf-8') as f:
    train_data = json.load(f)

In [3]:
for ind, data in train_data.items():
    query = data['query']
    if 'document' in data:
        document = data['document']
        if len(document) == 0:
            break

In [6]:
", ".join(train_data['400']['facet']), ", ".join(train_data['500']['facet'])

('beef, butternut squash, spaghetti squash, broccoli, broccoli',
 '1016 lofts, 1016 house, 1016 apartments')

In [7]:
train_data['500']

{'query': '1016',
 'facet': ['1016 lofts', '1016 house', '1016 apartments'],
 'document': ['Year 1016 was a leap year starting on Sunday (link will display the full calendar) of the Julian calendar. Events By place Europe. March 25 – Battle of Nesjar (off the coast of Norway): Olaf Haraldsson is victorious over former co-regent Sweyn ...',
  'At the time, the police used ten codes in their radio communication to road patrol officers. A 10-16 meant there was an open door at the scene. The street address of our house had that same number. So it became the 1016 Home - always having an open door to anyone who was wanting a life of recovery from alcohol and other drugs.',
  'Year 1016 was a leap year starting on Sunday (link will display the full calendar) of the Julian calendar.. Events By place Asia. March 10 – Emperor Sanjō gives up the throne of Japan and soon afterwards is replaced by his cousin Emperor Go-Ichijō, aged about 8. Earthquakes partly destroy the Dome of the Rock in the Old

In [6]:
# import re
# text = ""
# for sinppet in train_data['400']['document']:
#     text += sinppet + '\n' 
# text = text.strip()

# def remove_tag(content):
#     cleanr =re.compile('<.*?>')
#     cleantext = re.sub(cleanr, '', content)
#     return cleantext

# def cleanhtml(raw_html):
#     cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
#     cleantext = re.sub(cleanr, '', raw_html)
#     return cleantext

# # re.sub('<[^<]+?>', '', text)
# print(cleanhtml(text))

In [7]:
# print(remove_tag(text))

In [8]:
import json
with open("data/test.json", 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [9]:
test_data['0']

{'query': 'caesars atlantic city',
 'facet': ['caesars atlantic city events',
  'caesars atlantic city jobs',
  'caesars atlantic city parking'],
 'options_overall_label': 1,
 'document': ['Caesars Atlantic City Disappointed, frustrated, miserable Worst hotel experience My kids wanted to experience Atlantic City so book Caesar’s base on advertising it was a nightmare for my kids, room and bathroom nasty, holes in rags and towels dingy like been using for the past...',
  'Now $55 (Was $̶1̶1̶4̶) on Tripadvisor: Caesars Atlantic City, Atlantic City. See 18,311 traveler reviews, 1,818 candid photos, and great deals for Caesars Atlantic City, ranked #3 of 50 hotels in Atlantic City and rated 4 of 5 at Tripadvisor.',
  'Book the Caesars Atlantic City Resort  Casino - Stay at this 4-star beach resort in Atlantic City. Enjoy free WiFi, an outdoor pool, and 10 restaurants. Our guests praise the restaurant and the comfy rooms in our reviews. Popular attractions Ballys Atlantic City Casino and Bo

In [1]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/bart-base"
# model_path = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path) # facebook/bart-base

/home1/jovyan/venv_main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer.add_special_tokens({'cls_token': '[CLS]', 'pad_token': '[PAD]'})
special_tokens_dict = {'additional_special_tokens': ['[document]','[related]','[information]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50268, 768)

In [3]:
num_added_toks

3

In [27]:
input_string = ["caesars atlantic city", "100 best weight watchers recipes"]
# input_string = "caesars atlantic city"
enc_input = tokenizer(input_string, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors='pt')

taget_string = ["caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking", "beef, butternut squash, spaghetti squash, broccoli, broccoli"]
# taget_string = "caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking"
dec_input = tokenizer(taget_string, padding=True, truncation=True, max_length=tokenizer.model_max_length, return_tensors='pt')

In [28]:
enc_input['input_ids'].shape, dec_input['input_ids'].shape
tokenizer.decode(enc_input['input_ids'][0]), tokenizer.decode(dec_input['input_ids'][0])

('<s>caesars atlantic city</s>',
 '<s>caesars atlantic city events, caesars atlantic city jobs, caesars atlantic city parking</s>')

In [47]:
enc_input_ids = enc_input['input_ids']
enc_attention_mask = enc_input['attention_mask']

dec_input_ids = dec_input['input_ids']
labels = dec_input_ids[:,:-1].clone()
dec_input_ids = dec_input_ids[:,1:]
dec_attention_mask = dec_input['attention_mask'][:,1:]

In [36]:
dec_input_ids.shape, dec_attention_mask.shape, labels.shape

(torch.Size([2, 27]), torch.Size([2, 27]), torch.Size([2, 27]))

In [48]:
outputs = model(
    input_ids=enc_input_ids,
    attention_mask=enc_attention_mask,
    decoder_input_ids=dec_input_ids,
    decoder_attention_mask=dec_attention_mask,
    labels=labels
)

In [15]:
outputs.keys()

odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state'])

In [16]:
outputs.logits.shape

torch.Size([2, 27, 50265])

torch.Size([2, 27])

In [25]:
# test_string = "caesars atlantic city:"
# inputs = tokenizer([test_string], max_length=1024, return_tensors="pt")

# gen_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
# tokenizer.batch_decode(gen_ids, skip_special_tokens=True)[0]

## 파일로 로딩

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model_path = "/home/jovyan/hdfs-jmt-rungjoo-private/huggingface_models/bart-base"
# model_path = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path) # facebook/bart-base

In [2]:
from dataloader import data_loader
dataset = data_loader("data/test.json", model_path)

In [3]:
dataset[0]

{'query': 'caesars atlantic city',
 'facet': ['caesars atlantic city events',
  'caesars atlantic city jobs',
  'caesars atlantic city parking'],
 'options_overall_label': 1,
 'document': ['Caesars Atlantic City Disappointed, frustrated, miserable Worst hotel experience My kids wanted to experience Atlantic City so book Caesar’s base on advertising it was a nightmare for my kids, room and bathroom nasty, holes in rags and towels dingy like been using for the past...',
  'Now $55 (Was $̶1̶1̶4̶) on Tripadvisor: Caesars Atlantic City, Atlantic City. See 18,311 traveler reviews, 1,818 candid photos, and great deals for Caesars Atlantic City, ranked #3 of 50 hotels in Atlantic City and rated 4 of 5 at Tripadvisor.',
  'Book the Caesars Atlantic City Resort  Casino - Stay at this 4-star beach resort in Atlantic City. Enjoy free WiFi, an outdoor pool, and 10 restaurants. Our guests praise the restaurant and the comfy rooms in our reviews. Popular attractions Ballys Atlantic City Casino and Bo

In [4]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=dataset.collate_fn)

In [5]:
for line in dataloader:
    enc_input_ids, enc_attention_mask, dec_input_ids, dec_attention_mask, labels = line
    break

In [6]:
outputs = model(
    input_ids=enc_input_ids,
    attention_mask=enc_attention_mask,
    decoder_input_ids=dec_input_ids,
    decoder_attention_mask=dec_attention_mask,
    labels=labels
)

In [8]:
outputs.keys()

odict_keys(['loss', 'logits', 'encoder_last_hidden_state'])

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [13]:
enc_input_ids = enc_input_ids.to(device)
enc_attention_mask.to(device)
dec_input_ids.to(device)
dec_attention_mask.to(device)
labels.to(device)

tensor([[    0,   366,  1999,  3471,     6, 33141,   118,  3471,     6, 12668,
             6,  5565,     2,     1],
        [    0,  3209,   857,  4334, 15750,    13,  1392,     6,   364, 18707,
          1637, 15750,    13,  1392]], device='cuda:0')

In [14]:
enc_input_ids

tensor([[    0,  9502,  2887, 14506,     2,     1,     1],
        [    0,  3209,   857, 15750,    13,  1392,     2]], device='cuda:0')